In [2]:
from model.modules.fuseModel import FuseEncoder 
from lavis.datasets.builders import load_dataset
from model.modules.fuseModel import LavisEncoder, CLIPEncoder 
from utils.data_utils import get_fused_dataloader 
from lavis.models import load_model_and_preprocess
from hyptorch.lorentz.manifold import CustomLorentz
from transformers import AutoProcessor 
from config import parser
from typing import List
from config import CLIP_BASE_PATCH_16, CLIP_BASE_PATCH_32, COCO_PATH, FLICKR_PATH 

config = parser.parse_args()
manifold = CustomLorentz()

dataset = load_dataset("flickr30k", vis_path=FLICKR_PATH, cfg_path=None)
model_ckts = [CLIP_BASE_PATCH_16, CLIP_BASE_PATCH_32]

def prepare_processors(model_ckts:List[str]):
    tokenizers = []
    vis_processors = []
    txt_processors = []
    for model_ckt in model_ckts:
        if 'lavis' not in model_ckt:
            model, vis_processor, txt_processor = load_model_and_preprocess("blip_retrieval", "coco", is_eval=False)
            txt_processors.append(model.tokenizer)
            vis_processors.append(vis_processor['eval'])
            tokenizers.append(txt_processor['eval'])
        else:
            txt_processors.append(None)
            vis_processors.append(AutoProcessor.from_pretrained(model_ckt))
            tokenizers.append(AutoProcessor.from_pretrained(model_ckt))
    return tokenizers, vis_processors, txt_processors


tokenizers, vis_processors, txt_processor = prepare_processors(model_ckts)
        

loaders = get_fused_dataloader(dataset, vis_processors=vis_processors, tokenizers=tokenizers, txt_processors=txt_processors, batch_size=40) 

# model = FuseEncoder(
#     config,           
#     d_visions=[100, 100], 
#     d_texts=[100, 100], 
#     ft_out=256, 
#     vision_bodies=[], 
#     text_bodies=[],
#     vision_head=None,
#     text_head=None,
#     mapper=None,
#     manifold=None, 
# )

usage: ipykernel_launcher.py [-h] [--use_graph USE_GRAPH] [--lr LR]
                             [--dropout DROPOUT] [--cuda CUDA]
                             [--epochs EPOCHS] [--weight_decay WEIGHT_DECAY]
                             [--optimizer OPTIMIZER] [--momentum MOMENTUM]
                             [--patience PATIENCE] [--seed SEED]
                             [--log_freq LOG_FREQ] [--save SAVE]
                             [--save_dir SAVE_DIR] [--sweep_c SWEEP_C]
                             [--lr_reduce_freq LR_REDUCE_FREQ] [--gamma GAMMA]
                             [--grad_clip GRAD_CLIP] [--min_epochs MIN_EPOCHS]
                             [--mixed_precision MIXED_PRECISION]
                             [--gradient_accumulation_steps GRADIENT_ACCUMULATION_STEPS]
                             [--lorentz_pos_margin LORENTZ_POS_MARGIN]
                             [--lorentz_neg_margin LORENTZ_NEG_MARGIN]
                             [--lorentz_image_neg_margin LOREN

SystemExit: 2

/home/jarvis/anaconda3/envs/hada-v2/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


torch.Size([40, 12, 32])
torch.Size([40, 40, 12, 32])


In [35]:
sim_t2q = torch.matmul(
    a, b.permute(0, 2, 1)
).squeeze()

In [36]:
sim_t2q.max(-1).values.shape

torch.Size([40, 40])